In [1]:
import os
from pymongo import MongoClient
import bson
from datetime import datetime
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from sqlalchemy import create_engine, text
import pandas as pd
import os
import gc
import pickle

In [2]:
client = MongoClient('mongodb://localhost:27017/')

In [3]:
client.server_info()

In [4]:
!unzip sample_mflix.zip #un note these when turning in 

In [5]:
db = client.sample_mflix

In [6]:
# directory where the JSON files are stored
json_dir = 'sample_mflix'
json_files = [f for f in os.listdir(json_dir) if f.endswith(".json")]
collections = [f.replace(".json", "") for f in json_files]
collections

['users', 'theaters', 'movies', 'comments', 'sessions']

In [7]:
for idx, json_file in enumerate(json_files):
    with open(os.path.join(json_dir, json_file), 'r') as f:
        for line in f:
            data = bson.json_util.loads(line.strip())
            db[collections[idx]].insert_one(data)
        
        print(f"Loaded {json_file} into the '{collections[idx]}' collection.")

In [9]:
#q1
q1 = list(db.movies.find().limit(1))
with open('answers/q1.pkl', 'wb') as f:
   pickle.dump(q1, f)

# print(q1)

In [10]:
# ['_id', 'plot', 'genres', 'runtime', 'cast', 'num_mflix_comments', 
#  'title', 'fullplot', 'countries', 'released', 'directors', 'rated', 
#  'awards', 'lastupdated', 'year', 'imdb', 'type', 'tomatoes']

#these are the column names for movies

In [11]:
#q2
q2 = list(db.movies.find({"directors": "Christopher Nolan"}))
with open('answers/q2.pkl', 'wb') as f:
   pickle.dump(q2, f)

# print((q2))

In [12]:
#q3
q3 = list(db.users.find().limit(5))
with open('answers/q3.pkl', 'wb') as f:
   pickle.dump(q3, f)

# print(q3)


In [13]:
#q4
q4 = list(db.movies.find({
    "imdb.rating" : {"$gt" : 9}
}))
with open('answers/q4.pkl', 'wb') as f:
   pickle.dump(q4, f)

# print(q4)

In [14]:
#q5
q5 = db.movies.count_documents({})
with open('answers/q5.pkl', 'wb') as f:
   pickle.dump(q5, f)

# print(q5)

In [15]:
#q6
q2 = db.movies.count_documents({
    'year' : {'$gt' : 2000}
})
with open('answers/q2.pkl', 'wb') as f:
   pickle.dump(q2, f)

# print(q2)

In [16]:
#q7
q7 = db.movies.count_documents({
    'genres' : 'Comedy'
})
with open('answers/q7.pkl', 'wb') as f:
   pickle.dump(q7, f)

# print(q7)

# cursor = db.inspections.find({
#     "$or": [
#         { "address.city": "MANHTTAN" },
#         { "address.city": "BROOKLYN" }
#     ]
# })

# ny_la_inspections = list(cursor)
# ny_la_inspections

In [17]:
# p = db.comments.find_one()
# p

In [18]:
#q8
q8 = db.comments.count_documents({
    'name' : 'Taylor Hill'
})
with open('answers/q8.pkl', 'wb') as f:
   pickle.dump(q8, f)

# print(q8)

In [19]:
#q9
q9 = db.movies.count_documents({
    'runtime' : { '$gt' : 120}
})
with open('answers/q9.pkl', 'wb') as f:
   pickle.dump(q9, f)

# print(q9)

In [25]:
#q10
q10 = list(db.movies.find(
    {"year": 2015},    
    {"title": 1, "_id": 0}  
).sort("title", 1))     
with open('answers/q10.pkl', 'wb') as f:
   pickle.dump(q10, f)

# print(q10)

# inspections_2015 = db.inspections.find({
#     "date": {"$regex": "2015"}
# }).sort("id", 1)  # 1 for ascending order
# list(inspections_2015)

In [34]:
#q11
q11 = list(db.comments.find(
    {}  
).sort("date", -1).limit(5)) 
with open('answers/q11.pkl', 'wb') as f:
   pickle.dump(q11, f)

# print(q11)

In [40]:
#q12
q12 = db.movies.count_documents({ "$or" : [{"genres": "Action"},
                               {"genres" : "Adventure"}]})
with open('answers/q12.pkl', 'wb') as f:
   pickle.dump(q12, f)

# print(q12)

In [46]:
#q13
q13 = list(db.movies.find({"imdb.rating": {"$gt" : 9, "$lt" : 10}}))
with open('answers/q13.pkl', 'wb') as f:
   pickle.dump(q13, f)

# print(q13)

# cursor = db.trips.find({
#     "tripduration": {"$gte": 200, "$lte": 4000}, 
#     "gender": 1
# })
# trips = list(cursor)
# trips[:5]

In [48]:
#q14
q14 = db.movies.count_documents({"directors": { "$exists":True, "$size": 3 }})

with open('answers/q14.pkl', 'wb') as f:
    pickle.dump(q14, f)

# print(q14)

In [54]:
#q15
q15 = db.movies.count_documents({"directors":{"$exists":True},"$expr":{"$gte":[{"$size": "$directors" }, 3]}})
with open('answers/q2.pkl', 'wb') as f:
   pickle.dump(q2, f)

# print(q15)

In [67]:
#q16
pipeline = [{"$lookup": {"from": "movies","localField": "movie_id","foreignField": "_id",         
            "as": "movie_info"}},{"$unwind": "$movie_info"},{"$match": {"movie_info.year": 2010}},{"$count": "total"}]
q16 = list(db.comments.aggregate(pipeline))
with open('answers/16.pkl', 'wb') as f:
   pickle.dump(q16, f)

print(q16)

[{'total': 510}]


In [74]:
# q17
# q17 = db.comments.aggregate([{"$lookup": {"from": "movies","localField": "movie_id","foreignField": "_id",
#             "as": "movie_info"}},{"$unwind": "$movie_info"},{"$match": {"movie_info.genres": "action"}},{"$group": {"_id": "$username"}},{"$sort": "_id": 1
#         },{"$project": {"username": "$_id","_id": 0}}])
# q17 = [user["username"] for user in unique_users]

# with open('answers/q17.pkl', 'wb') as f:
#     pickle.dump(q17, f)

# # Print the result
# print(q17)


SyntaxError: invalid syntax (3585953254.py, line 3)

In [78]:
#q18
date = datetime(2018,8,13)
q18 = list(db.comments.find({"date": {'$gte':date}}))
with open('answers/q18.pkl', 'wb') as f:
   pickle.dump(q18, f)

# print(q18)

In [84]:
#q19
q19 = db.movies.find({"directors": "Steven Spielberg"},{'_id':1})
q19 = [movie['_id'] for movie in q19]
q19 = db.comments.count_documents({'movie_id' : {'$in':q19}})
with open('answers/q19.pkl', 'wb') as f:
   pickle.dump(q19, f)

# print(q19)

In [87]:
#q20
q20 = list(db.movies.find(
    {"imdb.rating": {'$gte': 9.0}},{'title': 1, 'released': 1, 'imdb.rating': 1, '_id': 0} 
).sort("released", -1).limit(10)) 
with open('answers/q20.pkl', 'wb') as f:
   pickle.dump(q20, f)

# print(q20)

In [95]:
#q21
total = db.movies.count_documents({})
movie_comments = db.comments.distinct("movie_id")
q21 = total - len(movie_comments)

with open('answers/q21.pkl', 'wb') as f:
   pickle.dump(q21, f)

# print(q21)

In [99]:
#q22
q22 = db.movies.count_documents({"title": {"$regex": "^The", "$options": "i"}})
with open('answers/q22.pkl', 'wb') as f:
   pickle.dump(q22, f)

# print(q22)

In [102]:
#q23
q23 = list(db.movies.aggregate([{"$match": {"genres": "Action"}},{"$group": {
        "_id": None,"average_rating": {"$avg": "$imdb.rating"}}},{"$project": {
        "_id": 0,"average_rating": {"$round": ["$avg_rating", 2]}}}]))
with open('answers/q23.pkl', 'wb') as f:
   pickle.dump(q23, f)

print(q23)

[{'average_rating': None}]


In [ ]:
#q24
q2 = list(db.movies.find({"directors": "Christopher Nolan"}))
with open('answers/q2.pkl', 'wb') as f:
   pickle.dump(q2, f)

# print(len(q2))

In [ ]:
#q25
q2 = list(db.movies.find({"directors": "Christopher Nolan"}))
with open('answers/q2.pkl', 'wb') as f:
   pickle.dump(q2, f)

# print(len(q2))

In [ ]:
#q26
q2 = list(db.movies.find({"directors": "Christopher Nolan"}))
with open('answers/q2.pkl', 'wb') as f:
   pickle.dump(q2, f)

# print(len(q2))

In [ ]:
#q27
q2 = list(db.movies.find({"directors": "Christopher Nolan"}))
with open('answers/q2.pkl', 'wb') as f:
   pickle.dump(q2, f)

# print(len(q2))

In [ ]:
#q28
q2 = list(db.movies.find({"directors": "Christopher Nolan"}))
with open('answers/q2.pkl', 'wb') as f:
   pickle.dump(q2, f)

# print(len(q2))

In [ ]:
#q29
q2 = list(db.movies.find({"directors": "Christopher Nolan"}))
with open('answers/q2.pkl', 'wb') as f:
   pickle.dump(q2, f)

# print(len(q2))

In [ ]:
#q30
q2 = list(db.movies.find({"directors": "Christopher Nolan"}))
with open('answers/q2.pkl', 'wb') as f:
   pickle.dump(q2, f)

# print(len(q2))

In [ ]:
#q31
q2 = list(db.movies.find({"directors": "Christopher Nolan"}))
with open('answers/q2.pkl', 'wb') as f:
   pickle.dump(q2, f)

# print(len(q2))

In [ ]:
#q32
q2 = list(db.movies.find({"directors": "Christopher Nolan"}))
with open('answers/q2.pkl', 'wb') as f:
   pickle.dump(q2, f)

# print(len(q2))

In [ ]:
#q33
q2 = list(db.movies.find({"directors": "Christopher Nolan"}))
with open('answers/q2.pkl', 'wb') as f:
   pickle.dump(q2, f)

# print(len(q2))

In [ ]:
#q34
q2 = list(db.movies.find({"directors": "Christopher Nolan"}))
with open('answers/q2.pkl', 'wb') as f:
   pickle.dump(q2, f)

# print(len(q2))